# Particle Filter

> Monte Carlo methods for recursive Bayesian filtering in non-linear, non-Gaussian systems

In [ ]:
#| default_exp rbe.particle_filter

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Optional, Callable, Tuple, List
from fastcore.test import test_eq, test_close
from fastcore.all import *
from technical_blog.rbe.probability import normalize, sample, eff_size

## Core Particle Filter Operations

In [ ]:
#| export
def init(n_particles, state_dim, init_fn=None, rng=None):
    "Initialize particle filter with `n_particles` and `state_dim`"
    if rng is None: rng = np.random.default_rng()
    
    if init_fn is None:
        # Default: uniform initialization in [0, 1]
        particles = rng.uniform(0, 1, size=(n_particles, state_dim))
    else:
        particles = init_fn(n_particles, state_dim, rng)
    
    weights = np.ones(n_particles) / n_particles
    return particles, weights

In [ ]:
# Test initialization
rng = np.random.default_rng(42)
particles, weights = init(100, 2, rng=rng)
assert particles.shape == (100, 2)
assert len(weights) == 100
test_close(np.sum(weights), 1.0)

## Predict and Update Steps

In [ ]:
#| export
def predict(particles, weights, transition_fn, rng=None):
    "Prediction step: apply `transition_fn` to `particles`"
    if rng is None: rng = np.random.default_rng()
    
    new_particles = np.zeros_like(particles)
    for i, particle in enumerate(particles):
        new_particles[i] = transition_fn(particle, rng)
    
    return new_particles, weights  # Weights unchanged in prediction

def update(particles, weights, observation, likelihood_fn):
    "Update step: weight `particles` using `observation` and `likelihood_fn`"
    new_weights = np.zeros_like(weights)
    
    for i, particle in enumerate(particles):
        new_weights[i] = weights[i] * likelihood_fn(particle, observation)
    
    # Normalize weights
    if np.sum(new_weights) > 0:
        new_weights = normalize(new_weights)
    else:
        # If all weights are zero, reset to uniform
        new_weights = np.ones_like(weights) / len(weights)
    
    return particles, new_weights

In [ ]:
# Test predict and update
def simple_transition(particle, rng):
    return particle + rng.normal(0, 0.1, size=particle.shape)

def simple_likelihood(particle, observation):
    # Gaussian likelihood
    diff = np.linalg.norm(particle - observation)
    return np.exp(-0.5 * diff**2)

# Test prediction
new_particles, new_weights = predict(particles, weights, simple_transition, rng)
assert new_particles.shape == particles.shape
test_close(new_weights, weights)  # Weights unchanged

# Test update
observation = np.array([0.5, 0.5])
particles, weights = update(particles, weights, observation, simple_likelihood)
test_close(np.sum(weights), 1.0)

## Resampling Methods

In [ ]:
#| export
def resample(particles, weights, method='systematic', rng=None):
    "Resample `particles` using `weights` with specified `method`"
    if rng is None: rng = np.random.default_rng()
    n_particles = len(particles)
    
    if method == 'systematic':
        # Systematic resampling
        positions = (np.arange(n_particles) + rng.uniform()) / n_particles
        cum_weights = np.cumsum(weights)
        indices = np.searchsorted(cum_weights, positions)
    elif method == 'multinomial':
        # Multinomial resampling
        indices = sample(weights, n_particles, rng)
    elif method == 'stratified':
        # Stratified resampling
        positions = (np.arange(n_particles) + rng.uniform(size=n_particles)) / n_particles
        cum_weights = np.cumsum(weights)
        indices = np.searchsorted(cum_weights, positions)
    else:
        raise ValueError(f"Unknown resampling method: {method}")
    
    new_particles = particles[indices]
    new_weights = np.ones(n_particles) / n_particles
    
    return new_particles, new_weights

In [ ]:
# Test resampling methods
for method in ['systematic', 'multinomial', 'stratified']:
    resampled_particles, resampled_weights = resample(particles, weights, method=method, rng=rng)
    assert resampled_particles.shape == particles.shape
    test_close(np.sum(resampled_weights), 1.0)
    test_close(resampled_weights, np.ones(100)/100)  # Should be uniform after resampling

## Complete Particle Filter Step

In [ ]:
#| export
def step(particles, weights, observation, transition_fn, likelihood_fn, 
         resample_threshold=0.5, rng=None):
    "Complete particle filter step: predict, update, and conditionally resample"
    # Prediction
    particles, weights = predict(particles, weights, transition_fn, rng)
    
    # Update
    particles, weights = update(particles, weights, observation, likelihood_fn)
    
    # Conditional resampling
    if eff_size(weights) < resample_threshold * len(particles):
        particles, weights = resample(particles, weights, rng=rng)
    
    return particles, weights

In [ ]:
# Test complete step
particles, weights = init(100, 2, rng=rng)
observation = np.array([0.5, 0.5])

particles, weights = step(particles, weights, observation, 
                         simple_transition, simple_likelihood, rng=rng)

assert particles.shape == (100, 2)
test_close(np.sum(weights), 1.0)

## Particle Filter Runner

High-level interface for running particle filters on sequences of observations.

In [ ]:
#| export
def run(observations, transition_fn, likelihood_fn, 
        n_particles=1000, init_fn=None, resample_threshold=0.5, rng=None):
    "Run particle filter on sequence of `observations`"
    if rng is None: rng = np.random.default_rng()
    
    # Infer state dimension from first observation
    state_dim = len(observations[0]) if hasattr(observations[0], '__len__') else 1
    
    # Initialize particles
    particles, weights = init(n_particles, state_dim, init_fn, rng)
    
    # Store results
    estimates = []
    particle_history = [particles.copy()]
    weight_history = [weights.copy()]
    eff_sizes = [eff_size(weights)]
    
    for obs in observations:
        # Particle filter step
        particles, weights = step(particles, weights, obs, 
                                 transition_fn, likelihood_fn, 
                                 resample_threshold, rng)
        
        # Estimate (weighted mean)
        estimate = np.average(particles, weights=weights, axis=0)
        estimates.append(estimate)
        
        # Store history
        particle_history.append(particles.copy())
        weight_history.append(weights.copy())
        eff_sizes.append(eff_size(weights))
    
    return {
        'estimates': np.array(estimates),
        'particles': particle_history,
        'weights': weight_history,
        'eff_sizes': np.array(eff_sizes)
    }

In [ ]:
# Test particle filter runner
# Generate synthetic data
true_states = [np.array([i * 0.1, i * 0.05]) for i in range(10)]
observations = [state + rng.normal(0, 0.1, 2) for state in true_states]

# Run filter
result = run(observations, simple_transition, simple_likelihood, 
             n_particles=100, rng=rng)

assert result['estimates'].shape == (10, 2)
assert len(result['particles']) == 11  # Initial + 10 steps
assert len(result['weights']) == 11
assert len(result['eff_sizes']) == 11

## Advanced Particle Filters

In [ ]:
#| export
def auxiliary_pf_step(particles, weights, observation, transition_fn, 
                     likelihood_fn, auxiliary_fn=None, rng=None):
    "Auxiliary particle filter step for improved proposal distribution"
    if rng is None: rng = np.random.default_rng()
    if auxiliary_fn is None:
        auxiliary_fn = likelihood_fn  # Default to standard PF
    
    n_particles = len(particles)
    
    # Compute auxiliary weights
    aux_weights = np.zeros(n_particles)
    for i, particle in enumerate(particles):
        # Look-ahead: what would the likelihood be after transition?
        predicted = transition_fn(particle, rng)
        aux_weights[i] = weights[i] * auxiliary_fn(predicted, observation)
    
    # Resample based on auxiliary weights
    if np.sum(aux_weights) > 0:
        aux_weights = normalize(aux_weights)
        indices = sample(aux_weights, n_particles, rng)
        particles = particles[indices]
    
    # Standard predict and update
    return step(particles, np.ones(n_particles)/n_particles, observation,
                transition_fn, likelihood_fn, resample_threshold=2.0, rng=rng)

In [ ]:
# Test auxiliary particle filter
particles, weights = init(100, 2, rng=rng)
particles, weights = auxiliary_pf_step(particles, weights, observation,
                                      simple_transition, simple_likelihood, rng=rng)
assert particles.shape == (100, 2)
test_close(np.sum(weights), 1.0)

## Export

In [ ]:
#| export
__all__ = [
    # Core operations
    'init', 'predict', 'update', 'resample', 'step',
    
    # High-level interface
    'run',
    
    # Advanced filters
    'auxiliary_pf_step'
]